In [197]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import check_nan, check_unique
from matplotlib import gridspec
from matplotlib import rcParams
from collections import Counter

from datetime import datetime, timedelta


import requests
import urllib.parse
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import geopy.distance
from geopy.distance import distance
#from mpl_toolkits.basemap import Basemap

#Paramètres graphiques 
rcParams['figure.figsize'] = 15,8 #Taille de la figure affichée
sns.set_style("darkgrid")#style de l'arrière plan de seaborn 
sns.set_palette("pastel")#Couleurs utilisées dans les graphiques 
plt.rcParams["figure.dpi"] = 200 #dpi = dot per inch , résolution des graphiques 
pd.set_option('display.max_columns', 500) #Nombre de colonne maximum du dataframe à afficher

# import warnings
# warnings.simplefilter(action='ignore')

In [198]:
data=pd.read_csv("./data/delays.csv")
data.columns= ["save_datetime","id_train_complet","transporteur","date_depart","depart_destination","arrivee_prevue","retard","nom_station"]

data["save_datetime"]=pd.to_datetime(data["save_datetime"])
data["date_depart"]=pd.to_datetime(data["date_depart"])


#Drop des lignes avec une horaire prévue inexploitatble :
df=data[data["arrivee_prevue"]!='Nie dotyczy']





#Conversion du retard en entier : 
df["retard"]=df["retard"].str.split(" ").str[0].astype('int')

#df[["depart","destination"]]= df["depart_destination"].str.split(" - ", expand=True)

new_id=df["id_train_complet"].str.extract(r"(\d+)(\/\d+)?\s*(\(\d+\)|)?(\s*)(.*|[a-zA-Z\u00C0-\u017F]*)").add_prefix('index_part')
df=df.merge(new_id, left_index=True, right_index=True).drop(['index_part3'], axis=1).rename(columns={"index_part4": "index_part3","index_part0":"id_train"})

df=df.fillna("")

#On ne garde que les lignes qui nous intéressent 
df =df.sort_values("save_datetime").drop_duplicates(["id_train","transporteur","date_depart","depart_destination","arrivee_prevue","nom_station"],keep="last").sort_values("save_datetime")

#df["arrivee_prevue"]=pd.to_datetime(df["arrivee_prevue"], format='%H:%M').dt.time

#df["arrivee_prevue"]= pd.Timestamp(df["arrivee_prevue"])

#df["save_datetime"]=df["save_datetime"].dt.strftime('%Y-%m-%d %H:%M:%S')

df["datee"]=df["save_datetime"].astype(str).str[:10]
df["arrivee_prevue2"]=pd.to_datetime(df["datee"]+" "+df["arrivee_prevue"], format='%Y-%m-%d %H:%M:%S')
df["arrivee_prevue"]=pd.to_datetime(df["arrivee_prevue"], format='%H:%M')

df['time_added'] = pd.to_timedelta(df["retard"],'m')
df["arrive"] =df["arrivee_prevue2"] + df['time_added']
df["save_datetime"]=df["save_datetime"].dt.strftime('%Y-%m-%d %H:%M:%S')

df["saved_datetime"]=df["save_datetime"].apply(pd.to_datetime)
df["delta"]=df["arrive"]-df["saved_datetime"]



df=df.drop(['saved_datetime','datee',"time_added","arrivee_prevue"], axis=1)


df["save_datetime"]=pd.to_datetime(df["save_datetime"])

df.loc[df["delta"]>timedelta(days=1),["arrive"]]=df.loc[df["delta"]>timedelta(days=1),["arrive"]] - timedelta(days= 1)
df.loc[df["delta"]>timedelta(days=1),["arrivee_prevue2"]]=df.loc[df["delta"]>timedelta(days=1),["arrivee_prevue2"]] - timedelta(days= 1)

df["saved_datetime"]=df["save_datetime"].apply(pd.to_datetime)
df["delta"]=df["arrive"]-df["saved_datetime"]
df = df[~(abs(df["delta"])>timedelta(minutes=10))]

df.drop(columns=["delta","saved_datetime"],inplace=True)

print("Shape du dataset d'origine :", data.shape)
print("Shape du dataset contenant uniquement les lignes qui nous intéressent : ",df.shape)
df.head()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Shape du dataset d'origine : (3718170, 8)
Shape du dataset contenant uniquement les lignes qui nous intéressent :  (223176, 13)


,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,retard,nom_station,id_train,index_part1,index_part2,index_part3,arrivee_prevue2,arrive
175,2022-05-16,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,24,Reda Pieleszewo,95697,,,,2022-05-15 23:38:00,2022-05-16 00:02:00
68,2022-05-16,91888/9,Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,2,Warszawa Stadion,91888,/9,,,2022-05-16 00:01:00,2022-05-16 00:03:00
111,2022-05-16,59486,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdynia Cisowa - Gdańsk Główny,9,Gdańsk Żabianka-AWFiS,59486,,,,2022-05-15 23:54:00,2022-05-16 00:03:00
119,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,0,Gdynia Wzgórze Św.Maksymiliana,95699,,,,2022-05-16 00:04:00,2022-05-16 00:04:00
116,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,0,Gdynia Redłowo,95699,,,,2022-05-16 00:01:00,2022-05-16 00:01:00


In [199]:
df.dtypes

save_datetime         datetime64[ns]
id_train_complet              object
transporteur                  object
date_depart           datetime64[ns]
depart_destination            object
retard                         int32
nom_station                   object
id_train                      object
index_part1                   object
index_part2                   object
index_part3                   object
arrivee_prevue2       datetime64[ns]
arrive                datetime64[ns]
dtype: object

In [200]:
check_nan(df)

Valeurs nan dans save_datetime : 0
Valeurs nan dans id_train_complet : 0
Valeurs nan dans transporteur : 0
Valeurs nan dans date_depart : 0
Valeurs nan dans depart_destination : 0
Valeurs nan dans retard : 0
Valeurs nan dans nom_station : 0
Valeurs nan dans id_train : 0
Valeurs nan dans index_part1 : 0
Valeurs nan dans index_part2 : 0
Valeurs nan dans index_part3 : 0
Valeurs nan dans arrivee_prevue2 : 0
Valeurs nan dans arrive : 0


# Mise en forme du df 

In [201]:
df[['depart','destination']] = df["depart_destination"].str.split(" - ",expand=True)

In [202]:
df.head()

,save_datetime,id_train_complet,transporteur,date_depart,depart_destination,retard,nom_station,id_train,index_part1,index_part2,index_part3,arrivee_prevue2,arrive,depart,destination
175,2022-05-16,95697,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdańsk Zaspa - Wejherowo,24,Reda Pieleszewo,95697,,,,2022-05-15 23:38:00,2022-05-16 00:02:00,Gdańsk Zaspa,Wejherowo
68,2022-05-16,91888/9,Koleje Mazowieckie,2022-05-16,Siedlce - Warszawa Zachodnia,2,Warszawa Stadion,91888,/9,,,2022-05-16 00:01:00,2022-05-16 00:03:00,Siedlce,Warszawa Zachodnia
111,2022-05-16,59486,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,Gdynia Cisowa - Gdańsk Główny,9,Gdańsk Żabianka-AWFiS,59486,,,,2022-05-15 23:54:00,2022-05-16 00:03:00,Gdynia Cisowa,Gdańsk Główny
119,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,0,Gdynia Wzgórze Św.Maksymiliana,95699,,,,2022-05-16 00:04:00,2022-05-16 00:04:00,Gdańsk Śródmieście,Wejherowo
116,2022-05-16,95699,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,Gdańsk Śródmieście - Wejherowo,0,Gdynia Redłowo,95699,,,,2022-05-16 00:01:00,2022-05-16 00:01:00,Gdańsk Śródmieście,Wejherowo


In [203]:
## Dataframe : 
df.columns = ["save_datetime","id_train_complet","transporteur","date_depart_trajet","depart_destination","retard",
              "station_pred","id_train","index_1","index_2","index_3","datetime_arrivee_prevue","arrivee_reelle_station_pred","depart","destination"]
cols_to_drop =["id_train_complet","depart_destination","index_1","index_2","index_3"]
df.drop(columns=cols_to_drop,inplace=True)
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination
175,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,24,Reda Pieleszewo,95697,2022-05-15 23:38:00,2022-05-16 00:02:00,Gdańsk Zaspa,Wejherowo
68,2022-05-16,Koleje Mazowieckie,2022-05-16,2,Warszawa Stadion,91888,2022-05-16 00:01:00,2022-05-16 00:03:00,Siedlce,Warszawa Zachodnia
111,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,9,Gdańsk Żabianka-AWFiS,59486,2022-05-15 23:54:00,2022-05-16 00:03:00,Gdynia Cisowa,Gdańsk Główny
119,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Wzgórze Św.Maksymiliana,95699,2022-05-16 00:04:00,2022-05-16 00:04:00,Gdańsk Śródmieście,Wejherowo
116,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Redłowo,95699,2022-05-16 00:01:00,2022-05-16 00:01:00,Gdańsk Śródmieście,Wejherowo


In [204]:
#Prediction P minutes avant : 
P= 55

### Ajout du timestamp où ce situe le train 

### Obtenir la dernière station 

In [205]:
df.iloc[0]

save_datetime                               2022-05-16 00:00:00
transporteur                   PKP SKM w Trójmieście Sp. z o.o.
date_depart_trajet                          2022-05-15 00:00:00
retard                                                       24
station_pred                                    Reda Pieleszewo
id_train                                                  95697
datetime_arrivee_prevue                     2022-05-15 23:38:00
arrivee_reelle_station_pred                 2022-05-16 00:02:00
depart                                             Gdańsk Zaspa
destination                                           Wejherowo
Name: 175, dtype: object

In [206]:
df["actual_datetime"]= df["datetime_arrivee_prevue"] -timedelta(minutes=P)

df["id_ad_train_nom_station"] = df["actual_datetime"].astype(str)+"_" + df["id_train"].astype(str)+"_"+df["station_pred"].astype(str)
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,id_ad_train_nom_station
175,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,24,Reda Pieleszewo,95697,2022-05-15 23:38:00,2022-05-16 00:02:00,Gdańsk Zaspa,Wejherowo,2022-05-15 22:43:00,2022-05-15 22:43:00_95697_Reda Pieleszewo
68,2022-05-16,Koleje Mazowieckie,2022-05-16,2,Warszawa Stadion,91888,2022-05-16 00:01:00,2022-05-16 00:03:00,Siedlce,Warszawa Zachodnia,2022-05-15 23:06:00,2022-05-15 23:06:00_91888_Warszawa Stadion
111,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,9,Gdańsk Żabianka-AWFiS,59486,2022-05-15 23:54:00,2022-05-16 00:03:00,Gdynia Cisowa,Gdańsk Główny,2022-05-15 22:59:00,2022-05-15 22:59:00_59486_Gdańsk Żabianka-AWFiS
119,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Wzgórze Św.Maksymiliana,95699,2022-05-16 00:04:00,2022-05-16 00:04:00,Gdańsk Śródmieście,Wejherowo,2022-05-15 23:09:00,2022-05-15 23:09:00_95699_Gdynia Wzgórze Św.Ma...
116,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Redłowo,95699,2022-05-16 00:01:00,2022-05-16 00:01:00,Gdańsk Śródmieście,Wejherowo,2022-05-15 23:06:00,2022-05-15 23:06:00_95699_Gdynia Redłowo


In [207]:
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,id_ad_train_nom_station
175,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,24,Reda Pieleszewo,95697,2022-05-15 23:38:00,2022-05-16 00:02:00,Gdańsk Zaspa,Wejherowo,2022-05-15 22:43:00,2022-05-15 22:43:00_95697_Reda Pieleszewo
68,2022-05-16,Koleje Mazowieckie,2022-05-16,2,Warszawa Stadion,91888,2022-05-16 00:01:00,2022-05-16 00:03:00,Siedlce,Warszawa Zachodnia,2022-05-15 23:06:00,2022-05-15 23:06:00_91888_Warszawa Stadion
111,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-15,9,Gdańsk Żabianka-AWFiS,59486,2022-05-15 23:54:00,2022-05-16 00:03:00,Gdynia Cisowa,Gdańsk Główny,2022-05-15 22:59:00,2022-05-15 22:59:00_59486_Gdańsk Żabianka-AWFiS
119,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Wzgórze Św.Maksymiliana,95699,2022-05-16 00:04:00,2022-05-16 00:04:00,Gdańsk Śródmieście,Wejherowo,2022-05-15 23:09:00,2022-05-15 23:09:00_95699_Gdynia Wzgórze Św.Ma...
116,2022-05-16,PKP SKM w Trójmieście Sp. z o.o.,2022-05-16,0,Gdynia Redłowo,95699,2022-05-16 00:01:00,2022-05-16 00:01:00,Gdańsk Śródmieście,Wejherowo,2022-05-15 23:06:00,2022-05-15 23:06:00_95699_Gdynia Redłowo


In [208]:
dict_map_last_station ={}
dict_map_arrivee_reelle ={}
dict_map_retard ={}
dict_map_arrivee_prevue={}

df_temp1 = df.copy(deep=True)
for id_train in tqdm(df["id_train"].unique()): 
    
    df_temp =df_temp1[df_temp1["id_train"]==id_train].drop_duplicates("arrivee_reelle_station_pred").sort_values("arrivee_reelle_station_pred")
    df_temp["arrivee_reelle_station_pred2"] = df_temp["arrivee_reelle_station_pred"]
    df_temp = df_temp.set_index("arrivee_reelle_station_pred")[["station_pred","id_ad_train_nom_station","retard","arrivee_reelle_station_pred2","datetime_arrivee_prevue"]]
    
    index_date= pd.date_range(start=df_temp.index.min() - timedelta(minutes=P),
                  end = df_temp.index.max(), freq ="1T" )
    
    df_temp= df_temp.reindex(index_date,method="ffill" )
    
    df_temp["last_station"] = df_temp["station_pred"].shift(P)
    df_temp["retard"] = df_temp["retard"].shift(P)
    df_temp["arrivee_reelle_station_pred2"] = df_temp["arrivee_reelle_station_pred2"].shift(P)
    df_temp["datetime_arrivee_prevue"] = df_temp["datetime_arrivee_prevue"].shift(P)
    
    dict_map_last_station.update(df_temp.drop_duplicates(["id_ad_train_nom_station"],keep="first").set_index("id_ad_train_nom_station")["last_station"].to_dict())
    dict_map_arrivee_reelle.update(df_temp.drop_duplicates(["id_ad_train_nom_station"],keep="first").set_index("id_ad_train_nom_station")["arrivee_reelle_station_pred2"].to_dict())
    dict_map_retard.update(df_temp.drop_duplicates(["id_ad_train_nom_station"],keep="first").set_index("id_ad_train_nom_station")["retard"].to_dict())
    dict_map_arrivee_prevue.update(df_temp.drop_duplicates(["id_ad_train_nom_station"],keep="first").set_index("id_ad_train_nom_station")["datetime_arrivee_prevue"].to_dict())
    
df["last_station"] = df["id_ad_train_nom_station"].map(dict_map_last_station)
df["last_station_arrivee_prevue"] = df["id_ad_train_nom_station"].map(dict_map_arrivee_prevue)
df["last_station_arrivee_reelle"] = df["id_ad_train_nom_station"].map(dict_map_arrivee_reelle)
df["last_station_retard"] = df["id_ad_train_nom_station"].map(dict_map_retard)

100%|██████████| 3447/3447 [02:56<00:00, 19.47it/s]


In [209]:
df=df.dropna()

In [210]:
check_nan(df)
print()
check_unique(df)

Valeurs nan dans save_datetime : 0
Valeurs nan dans transporteur : 0
Valeurs nan dans date_depart_trajet : 0
Valeurs nan dans retard : 0
Valeurs nan dans station_pred : 0
Valeurs nan dans id_train : 0
Valeurs nan dans datetime_arrivee_prevue : 0
Valeurs nan dans arrivee_reelle_station_pred : 0
Valeurs nan dans depart : 0
Valeurs nan dans destination : 0
Valeurs nan dans actual_datetime : 0
Valeurs nan dans id_ad_train_nom_station : 0
Valeurs nan dans last_station : 0
Valeurs nan dans last_station_arrivee_prevue : 0
Valeurs nan dans last_station_arrivee_reelle : 0
Valeurs nan dans last_station_retard : 0

Valeurs uniques dans save_datetime : 3994
Valeurs uniques dans transporteur : 10
Valeurs uniques dans date_depart_trajet : 14
Valeurs uniques dans retard : 242
Valeurs uniques dans station_pred : 95
Valeurs uniques dans id_train : 3442
Valeurs uniques dans datetime_arrivee_prevue : 18600
Valeurs uniques dans arrivee_reelle_station_pred : 18697
Valeurs uniques dans depart : 277
Valeurs 

In [211]:
df.drop(columns=["id_ad_train_nom_station"],inplace=True)

## Ajout météo 

In [212]:
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,last_station,last_station_arrivee_prevue,last_station_arrivee_reelle,last_station_retard
2509,2022-05-16 01:15:00,PKP Intercity,2022-05-16,1,Zabrze,16170,2022-05-16 01:15:00,2022-05-16 01:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:20:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0
2629,2022-05-16 01:20:00,PKP Intercity,2022-05-16,1,Gliwice,16170,2022-05-16 01:23:00,2022-05-16 01:24:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:28:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0
3150,2022-05-16 01:45:00,PKP Intercity,2022-05-16,0,Poznań Główny,18170,2022-05-16 01:49:00,2022-05-16 01:49:00,Warszawa Wschodnia,Świnoujście,2022-05-16 00:54:00,Konin,2022-05-16 00:51:00,2022-05-16 00:51:00,0.0
3573,2022-05-16 02:00:00,PKP Intercity,2022-05-16,0,Legionowo,38170,2022-05-16 02:04:00,2022-05-16 02:04:00,Kraków Główny,Kołobrzeg,2022-05-16 01:09:00,Warszawa Zachodnia,2022-05-16 00:55:00,2022-05-16 00:55:00,0.0
3514,2022-05-16 02:00:00,PKP Intercity,2022-05-16,11,Poznań Główny,53172,2022-05-16 01:53:00,2022-05-16 02:04:00,Gdynia Główna,Kraków Główny,2022-05-16 00:58:00,Bydgoszcz Główna,2022-05-16 00:01:00,2022-05-16 00:06:00,5.0


In [213]:
def get_loc(loc_list):
    loc={}
    for i in loc_list:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote("Pologne "+i) + '?format=json'
        response = requests.get(url).json()
        if len(response)!=0:
            loc[i]=[float(response[0]['lon']),float(response[0]['lat'])]
        else:
            loc[i]=[None,None]
    return loc

In [214]:
dep_gare=set(df["depart"].unique())
arr_gare=set(df["destination"].unique())
gare=dep_gare.union(arr_gare)
gare_loc=get_loc(gare)

gare_loc['Loecknitz']=[14.2153,53.4528]
gare_loc['Tantow']=[14.3333,53.2667]
gare_loc['Jagodin']=[21.257272,43.977729]
gare_loc['Forst Lausitz']=[14.6474905,51.7441459]
gare_loc['Mezimesti']=[16.242078,50.624605]

In [215]:
big_city=pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Poland")[0]["Name"].to_list()
big_city=[i.split('(', 1)[0] for i in big_city]
big_loc=get_loc(big_city)

del big_loc["Grudziądz"]
del big_loc["Świdnica"]
del big_loc["Włocławek"]
del big_loc["Gniezno"]
del big_loc["Ostrowiec Świętokrzyski"]
del big_loc["Chełm"]
del big_loc["Bydgoszcz"]
del big_loc["Ełk"]
del big_loc["Wałbrzych"]
del big_loc["Inowrocław"]
del big_loc["Głogów"]

In [216]:
start = datetime(2022, 5, 16)
end = datetime(2022, 5, 30)

weather_gare={}
weather_gare_all=pd.DataFrame()

for i in gare_loc:

    point = Point(gare_loc[i][1],gare_loc[i][0])
    data = Daily(point, start, end)
    data = data.fetch()
    if len(data)>0:
        weather_gare[i]=data
        data["gare"]=i
        
        weather_gare_all=pd.concat([weather_gare_all,data])

    else:
        near_city=''
        min=999999999999999999999
        for city,coord in big_loc.items():
            d = distance((gare_loc[i][1],gare_loc[i][0]), (big_loc[city][1],big_loc[city][0])).km
            if d<min:
                min=d
                near_city=city
        point = Point(big_loc[near_city][1],big_loc[near_city][0])
        data = Daily(point, start, end)
        data = data.fetch()
        data["gare"]=i
        weather_gare[i]=data
        weather_gare_all=pd.concat([weather_gare_all, data])
        
weather_gare_all=weather_gare_all.fillna(0)
weather_gare_all=weather_gare_all.reset_index()


In [217]:
weather_gare_all["time"]=weather_gare_all["time"].astype('datetime64[ns]')
df["full_day"]=df["save_datetime"].dt.strftime('%Y-%m-%d').astype('datetime64[ns]')


In [218]:
df=df.merge(weather_gare_all, left_on=["full_day",'depart'], right_on=["time",'gare'])
df=df.drop(columns=['gare', 'time'])


df=df.merge(weather_gare_all, left_on=["full_day",'destination'], right_on=["time",'gare'],suffixes=('', '_d'))
df=df.drop(columns=['gare', 'time'])

df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,last_station,last_station_arrivee_prevue,last_station_arrivee_reelle,last_station_retard,full_day,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,tavg_d,tmin_d,tmax_d,prcp_d,snow_d,wdir_d,wspd_d,wpgt_d,pres_d,tsun_d
0,2022-05-16 01:15:00,PKP Intercity,2022-05-16,1,Zabrze,16170,2022-05-16 01:15:00,2022-05-16 01:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:20:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0
1,2022-05-16 01:20:00,PKP Intercity,2022-05-16,1,Gliwice,16170,2022-05-16 01:23:00,2022-05-16 01:24:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:28:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0
2,2022-05-16 04:15:00,PKP Intercity,2022-05-16,5,Wrocław Główny,16170,2022-05-16 04:11:00,2022-05-16 04:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 03:16:00,Gliwice,2022-05-16 01:23:00,2022-05-16 01:24:00,1.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0
3,2022-05-16 09:05:00,PKP Intercity,2022-05-16,0,Koluszki,16100,2022-05-16 09:06:00,2022-05-16 09:06:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 08:11:00,Warszawa Gdańska,2022-05-16 07:53:00,2022-05-16 07:56:00,3.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0
4,2022-05-16 10:20:00,PKP Intercity,2022-05-16,0,Częstochowa,16100,2022-05-16 10:23:00,2022-05-16 10:23:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 09:28:00,Koluszki,2022-05-16 09:06:00,2022-05-16 09:06:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0


## Distance géo 

In [219]:
all_combi=df[["station_pred","last_station"]].drop_duplicates(["station_pred","last_station"])


station_loc=get_loc(df["station_pred"].unique())

In [220]:
all_combi["dist"]=all_combi.apply(lambda x: distance((station_loc[x["station_pred"]][1],
                                       station_loc[x["station_pred"]][0]),
                                      (station_loc[x["last_station"]][1],
                                       station_loc[x["last_station"]][0])).km, axis=1)

all_combi.head()

,station_pred,last_station,dist
0,Zabrze,Zawiercie,49.989084
1,Gliwice,Zawiercie,58.552398
2,Wrocław Główny,Gliwice,145.675300
3,Koluszki,Warszawa Gdańska,98.880919
4,Częstochowa,Koluszki,114.802200


In [221]:
df=pd.merge(df, all_combi,  how='left', left_on=['station_pred','last_station'], right_on = ['station_pred','last_station'])
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,last_station,last_station_arrivee_prevue,last_station_arrivee_reelle,last_station_retard,full_day,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,tavg_d,tmin_d,tmax_d,prcp_d,snow_d,wdir_d,wspd_d,wpgt_d,pres_d,tsun_d,dist
0,2022-05-16 01:15:00,PKP Intercity,2022-05-16,1,Zabrze,16170,2022-05-16 01:15:00,2022-05-16 01:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:20:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,49.989084
1,2022-05-16 01:20:00,PKP Intercity,2022-05-16,1,Gliwice,16170,2022-05-16 01:23:00,2022-05-16 01:24:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:28:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,58.552398
2,2022-05-16 04:15:00,PKP Intercity,2022-05-16,5,Wrocław Główny,16170,2022-05-16 04:11:00,2022-05-16 04:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 03:16:00,Gliwice,2022-05-16 01:23:00,2022-05-16 01:24:00,1.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,145.675300
3,2022-05-16 09:05:00,PKP Intercity,2022-05-16,0,Koluszki,16100,2022-05-16 09:06:00,2022-05-16 09:06:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 08:11:00,Warszawa Gdańska,2022-05-16 07:53:00,2022-05-16 07:56:00,3.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,98.880919
4,2022-05-16 10:20:00,PKP Intercity,2022-05-16,0,Częstochowa,16100,2022-05-16 10:23:00,2022-05-16 10:23:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 09:28:00,Koluszki,2022-05-16 09:06:00,2022-05-16 09:06:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,114.802200


### Classico : 

In [222]:
df['day_now']=df["actual_datetime"].dt.day
df['dayofweek_num_now']=df["actual_datetime"].dt.dayofweek  
df['hour_now'] = df["actual_datetime"].dt.hour 
df['minute_now'] = df["actual_datetime"].dt.minute

## Distance temps minutes : 

In [223]:
df["delta_temps_stations"]=(df["datetime_arrivee_prevue"] - df["last_station_arrivee_prevue"]).dt.seconds//60

### Nombre de passage dans la gare : 

In [224]:
df['all_passa_station_d'] = df.groupby(["day_now","station_pred"]).cumcount()
df['all_passa_station_h'] = df.groupby(["hour_now","station_pred"]).cumcount()

In [225]:
df.shape

(207418, 44)

In [226]:
check_nan(df)
print()
check_unique(df)

Valeurs nan dans save_datetime : 0
Valeurs nan dans transporteur : 0
Valeurs nan dans date_depart_trajet : 0
Valeurs nan dans retard : 0
Valeurs nan dans station_pred : 0
Valeurs nan dans id_train : 0
Valeurs nan dans datetime_arrivee_prevue : 0
Valeurs nan dans arrivee_reelle_station_pred : 0
Valeurs nan dans depart : 0
Valeurs nan dans destination : 0
Valeurs nan dans actual_datetime : 0
Valeurs nan dans last_station : 0
Valeurs nan dans last_station_arrivee_prevue : 0
Valeurs nan dans last_station_arrivee_reelle : 0
Valeurs nan dans last_station_retard : 0
Valeurs nan dans full_day : 0
Valeurs nan dans tavg : 0
Valeurs nan dans tmin : 0
Valeurs nan dans tmax : 0
Valeurs nan dans prcp : 0
Valeurs nan dans snow : 0
Valeurs nan dans wdir : 0
Valeurs nan dans wspd : 0
Valeurs nan dans wpgt : 0
Valeurs nan dans pres : 0
Valeurs nan dans tsun : 0
Valeurs nan dans tavg_d : 0
Valeurs nan dans tmin_d : 0
Valeurs nan dans tmax_d : 0
Valeurs nan dans prcp_d : 0
Valeurs nan dans snow_d : 0
Vale

In [227]:
df.head()

,save_datetime,transporteur,date_depart_trajet,retard,station_pred,id_train,datetime_arrivee_prevue,arrivee_reelle_station_pred,depart,destination,actual_datetime,last_station,last_station_arrivee_prevue,last_station_arrivee_reelle,last_station_retard,full_day,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,tavg_d,tmin_d,tmax_d,prcp_d,snow_d,wdir_d,wspd_d,wpgt_d,pres_d,tsun_d,dist,day_now,dayofweek_num_now,hour_now,minute_now,delta_temps_stations,all_passa_station_d,all_passa_station_h
0,2022-05-16 01:15:00,PKP Intercity,2022-05-16,1,Zabrze,16170,2022-05-16 01:15:00,2022-05-16 01:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:20:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,49.989084,16,0,0,20,74,0,0
1,2022-05-16 01:20:00,PKP Intercity,2022-05-16,1,Gliwice,16170,2022-05-16 01:23:00,2022-05-16 01:24:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 00:28:00,Zawiercie,2022-05-16 00:01:00,2022-05-16 00:01:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,58.552398,16,0,0,28,82,0,0
2,2022-05-16 04:15:00,PKP Intercity,2022-05-16,5,Wrocław Główny,16170,2022-05-16 04:11:00,2022-05-16 04:16:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 03:16:00,Gliwice,2022-05-16 01:23:00,2022-05-16 01:24:00,1.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,145.675300,16,0,3,16,168,0,0
3,2022-05-16 09:05:00,PKP Intercity,2022-05-16,0,Koluszki,16100,2022-05-16 09:06:00,2022-05-16 09:06:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 08:11:00,Warszawa Gdańska,2022-05-16 07:53:00,2022-05-16 07:56:00,3.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,98.880919,16,0,8,11,73,0,0
4,2022-05-16 10:20:00,PKP Intercity,2022-05-16,0,Częstochowa,16100,2022-05-16 10:23:00,2022-05-16 10:23:00,Warszawa Wschodnia,Jelenia Góra,2022-05-16 09:28:00,Koluszki,2022-05-16 09:06:00,2022-05-16 09:06:00,0.0,2022-05-16,14.5,8.4,19.3,0.0,0.0,345.0,8.4,22.2,1020.3,0.0,12.2,4.5,18.8,0.0,0.0,194.0,6.4,27.8,1018.9,0.0,114.802200,16,0,9,28,77,0,0


In [228]:
df.to_csv("data_processed_cleaned_5 5min.csv",index=False)